In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
**Task Definition**
You are an **SMT expert** specializing in **Semantic Fusion**, a method for validating SMT solvers. Your task is to **automatically fuse two satisfiable SMT formulas** into a new formula
 while **preserving satisfiability.**

The formulas are written in **SMT-LIB syntax**. Your goal is to take two input formulas **ω1 and ω2**, perform **Semantic Fusion**, and return a **new formula ω12** that remains satisfiable by construction.

### SMT-LIB Syntax Guidelines:
**The generated output must strictly conform to the SMT-LIB format and include**:

1.	Variable Declarations: Use declare-fun to declare variables with their types: (declare-fun <var> () <type>).
2.	Function Definitions: Use define-fun to define functions: (define-fun <name> (<arg1 type1> <arg2 type2> ...) <return-type> <body>).
3.	Assertions: Use assert to specify constraints: (assert <constraint>).
4.	Boolean Expressions:
	•	Logical AND: (and <expr1> <expr2> ...)
	•	Logical OR: (or <expr1> <expr2> ...)
	•	Logical NOT: (not <expr>)
5.	Arithmetic Operations: Use standard operators like +, -, *, /, and div for integers.
6.	Relational Operators: Use comparison operators like >, <, >=, <=, and = for numerical expressions.
7.	Equality and Disequality: Use = for equality and distinct for disequality: (distinct <expr1> <expr2> ...).
8.	Conditionals: Use ite for if-then-else expressions: (ite <condition> <then-branch> <else-branch>).
9.	Quantifiers: Use forall and exists for quantified expressions:
	•	Universal: (forall ((x <type>) ...) <body>)
	•	Existential: (exists ((x <type>) ...) <body>)
10.	Constants: Declare constants directly using their types:
	•	Integers: 1, -3
	•	Real numbers: 1.0, -2.5
	•	Booleans: true, false
11.	Parentheses Matching: Ensure all expressions are enclosed in properly matched parentheses.
12. Ensure all arithmatic operations use the prefix notation (operator operand1 operand2 ... ). Example: (+ x y) (instead of x + y)
13.	Comments: Use ; for single-line comments in SMT-LIB.

**Avoid invalid SMT-LIB constructs! Ensure all parentheses are correctly matched.**

**Semantic Fusion Steps**:

1. Merge ω1 and ω2 into a single formula ω1 ∧ ω2, preserving original declarations and assertions.

2. Introduce a fresh variable z which is not present in either input formula.

3. Based on the datatypes of x and y from the table below, choose the fusion function and then derive the inversion functions rx(y, z) and ry(x, z) from it :
    

| Type      | Fusion Function f(x, y)                | r_x(y, z)                                |  r_y(x, z)                                     |
|-----------|----------------------------------------|------------------------------------------|------------------------------------------------|
| **Int**   |  x + y                                 | z - y                                    | z - x                                          |
|           |  x + c + y                             | z - c - y                                | z - c - x                                      |
|           |  x * y                                 | z div y                                  | z div x                                        |
|           |  c_1 * x + c_2 * y + c_3               | (z - c_2 * y - c_3) div c_1              | (z - c_1 * x - c_3) div c_2                    |
| **Real**  |  x + y                                 | z - y                                    | z - x                                          |
|           |  x + c + y                             | z - c - y                                | z - c - x                                      |
|           |  x * y                                 | z / y                                    | z / x                                          |
|           |  c_1 * x + c_2 * y + c_3               | (z - c_2 * y - c_3) / c_1                | (z - c_1 * x - c_3) / c_2                      |
| **String**|  x str++ y                             | str.substr(z, 0, str.len(x))             | str.substr(z, str.len(x), str.len(y))          |
|           |  x str++ y                             | str.substr z 0 (str.len x)               | str.replace z x ""                             |
|           |  x str++ c str++ y                     | str.substr(z, 0, str.len(x))             | str.replace(str.replace(z, x, ""), c, "")      |



4. Next, define the inversion functions. In the example they would be defined as followed:
        	•	rx(y, z)  (expression for x)
	        •	ry(x, z)  (expression for y)
5. Then, derive expressions for `rx(y, z)` and `ry(x, z)`. Make sure the expressions are correctly in prefix notation.

    
6.	Substitute variables:
	•	Randomly replace **one** free occurrence of x in ω1 **with the explicit expression value** of rx(y, z), in correct SMT-LIB syntax and **NOT rx(y, z) itself**.
	•	Randomly replace **one** free occurrence of y in ω2 **with the explicit expression value** of ry(y, z), in correct SMT-LIB syntax and **NOT rx(y, z) itself**.

7. **DO NOT introduce new assertions**—the fused formula should be **syntactically valid SMT-LIB** without modifying logical constraints.

8. Do NOT add any new assert blocks to the conjoined formula!


###Example

**Formula1(ω1)**:
(declare-fun x () Real)
(assert (or (> x 0) (> x 1)))

**Formula2(ω2)**:
(declare-fun y () Real)
(assert (or (< y 0) (< y 1)))

**Expected Output**

(declare-fun x () Real)
(declare-fun y () Real)
(declare-fun z () Real)

; Fusion: Introduce z = x * y
(assert (or 
    (or (> x 0) (> (z / y) 1)) ; Randomly replaced only one x with (z div y)
    (or (< y 0) (< (z / x) 1)) ; Randomly  replaced only one y with (z div x)
))

###Additional Constraints
- Provide the result after performing each step.
- **No new assertions should be added**, only substitutions.
- **Variable names must remain unchanged**, except for z.
- Refrain from inserting comments in the final fused formula.

Now, perform the described operation for these formulas:

        """
        ),
        (
            "human",
            """
ω1: {formula1}
ω2: {formula2}
"""
    ),
]
)

In [2]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="KM3PGpAYB10SjVRj5IH5mysWswYiMJ4N"
)
chain = prompt | llm

In [ ]:
from langchain_ollama import ChatOllama 

llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0
)

### Formulas to test SAT Fusion

In [ ]:
# Formulae from the SAT Fusion paper
chain = prompt | llm
formula1 = """
( declare-fun x () Int )
( declare-fun w () Bool )

( assert (= x (- 1) ) )
( assert (= w (= x (- 1) ) ) )
( assert w )
"""

formula2 = """
( declare-fun y () Int )
( declare-fun v () Bool )

( assert (= v ( not (= y (- 1) ) ) ) )
( assert ( ite v false (= y (- 1) ) ) )
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Propositional Logic (QF_BV) anfangen
formula1 = """
(set-logic QF_BV)
(declare-fun a () Bool)
(declare-fun b () Bool)
(declare-fun c () Bool)

(assert (or (and a b) (and (not a) c)))
(assert (or (not b) c))
(assert (xor a c))
(check-sat)
"""

formula2 = """
(set-logic QF_BV)
(declare-fun x () Bool)
(declare-fun y () Bool)
(declare-fun z () Bool)

(assert (or (and x y) (not z)))
(assert (or x (not y) z))
(assert (xor y z))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [4]:
# Propositional Logic (QF_BV) anfangen
formula1 = """
(set-logic QF_BV)
(declare-fun T1_21130 () (_ BitVec 8))
(declare-fun T1_21045 () (_ BitVec 8))
(declare-fun T1_20966 () (_ BitVec 8))
(declare-fun T1_20916 () (_ BitVec 8))
(declare-fun T1_20912 () (_ BitVec 8))
(declare-fun T2_20520 () (_ BitVec 16))
(declare-fun T1_20511 () (_ BitVec 8))
(declare-fun T2_20914 () (_ BitVec 16))
(declare-fun T1_20914 () (_ BitVec 8))
(declare-fun T1_20915 () (_ BitVec 8))
(declare-fun T1_20520 () (_ BitVec 8))
(declare-fun T1_20521 () (_ BitVec 8))
(assert (let ((?v_19 ((_ zero_extend 24) T1_21130)) (?v_15 ((_ zero_extend 24) T1_21045)) (?v_0 ((_ zero_extend 24) (_ bv1 8))) (?v_9 ((_ zero_extend 24) T1_20966)) (?v_6 ((_ zero_extend 24) T1_20916)) (?v_7 ((_ zero_extend 24) T1_20912)) (?v_3 ((_ zero_extend 16) T2_20520)) (?v_2 (bvsub ((_ zero_extend 24) T1_20511) (_ bv8 32)))) (let ((?v_4 (bvadd (bvadd (bvadd (bvadd ?v_2 (_ bv4271429 32)) ?v_3) (_ bv7 32)) ?v_7))) (let ((?v_8 (bvadd (bvadd ?v_4 (_ bv3 32)) ?v_6))) (let ((?v_13 (bvadd (bvadd ?v_8 ?v_0) ?v_9))) (let ((?v_1 (bvsub (bvadd (bvadd ?v_13 ?v_0) ?v_15) (_ bv4271399 32)))) (let ((?v_20 (bvsub (bvadd (bvadd ?v_1 (_ bv4271400 32)) ?v_19) (_ bv1 32))) (?v_18 ((_ zero_extend 16) T2_20914)) (?v_14 (bvsub ?v_13 (_ bv4271399 32)))) (let ((?v_16 (bvsub (bvadd (bvadd ?v_14 (_ bv4271400 32)) ?v_15) (_ bv1 32))) (?v_17 (bvadd ?v_14 (_ bv4271484 32))) (?v_10 (bvsub ?v_8 (_ bv4271399 32)))) (let ((?v_12 (bvsub (bvadd (bvadd ?v_10 (_ bv4271400 32)) ?v_9) (_ bv1 32))) (?v_11 (bvadd ?v_10 (_ bv4271478 32))) (?v_5 (bvsub ?v_4 (_ bv4271397 32)))) (and true (= T2_20520 (bvor (bvshl ((_ zero_extend 8) T1_20521) (_ bv8 16)) ((_ zero_extend 8) T1_20520))) (= T2_20914 (bvor (bvshl ((_ zero_extend 8) T1_20915) (_ bv8 16)) ((_ zero_extend 8) T1_20914))) (bvule ?v_20 (bvadd ?v_1 (_ bv4271462 32))) (not (= (bvadd (bvadd ?v_2 (_ bv29 32)) (_ bv4271400 32)) (_ bv0 32))) (not (= ?v_3 (_ bv0 32))) (bvult (bvadd ?v_5 (_ bv4271426 32)) (bvsub (bvadd (bvadd ?v_5 (_ bv4271400 32)) ?v_6) (_ bv1 32))) (= ?v_7 (_ bv1 32)) (not (= ?v_18 (_ bv0 32))) (bvule ?v_12 ?v_11) (not (= ?v_11 ?v_12)) (bvult (bvadd ?v_10 (_ bv4271462 32)) ?v_12) (not (= ?v_17 ?v_16)) (bvule ?v_16 ?v_17) (bvult (bvadd ?v_14 (_ bv4271460 32)) ?v_16) (bvult (bvadd ?v_14 (_ bv4271434 32)) ?v_16) (bvult (bvadd ?v_14 (_ bv4271418 32)) ?v_16) (= (bvadd (bvadd (bvadd (bvadd ?v_18 (bvsub (_ bv4294967295 32) ?v_6)) (bvsub (_ bv4294967295 32) ?v_9)) (bvsub (_ bv4294967295 32) ?v_15)) (bvsub (_ bv4294967295 32) ?v_19)) (_ bv0 32)) (bvult (bvadd ?v_1 (_ bv4271420 32)) ?v_20)))))))))))
(check-sat)
"""

formula2 = """
(set-logic QF_BV)
(set-info :category "industrial")
(set-info :status sat)
(declare-fun T1_21130 () (_ BitVec 8))
(declare-fun T1_21045 () (_ BitVec 8))
(declare-fun T1_20966 () (_ BitVec 8))
(declare-fun T1_20916 () (_ BitVec 8))
(declare-fun T2_20914 () (_ BitVec 16))
(declare-fun T1_20912 () (_ BitVec 8))
(declare-fun T2_20520 () (_ BitVec 16))
(declare-fun T1_20511 () (_ BitVec 8))
(declare-fun T1_20914 () (_ BitVec 8))
(declare-fun T1_20915 () (_ BitVec 8))
(declare-fun T1_20520 () (_ BitVec 8))
(declare-fun T1_20521 () (_ BitVec 8))
(assert (let ((?v_19 ((_ zero_extend 24) T1_21130)) (?v_15 ((_ zero_extend 24) T1_21045)) (?v_9 ((_ zero_extend 24) T1_20966)) (?v_6 ((_ zero_extend 24) T1_20916)) (?v_18 ((_ zero_extend 16) T2_20914)) (?v_0 ((_ zero_extend 24) (_ bv1 8))) (?v_7 ((_ zero_extend 24) T1_20912)) (?v_3 ((_ zero_extend 16) T2_20520)) (?v_2 (bvsub ((_ zero_extend 24) T1_20511) (_ bv8 32)))) (let ((?v_4 (bvadd (bvadd (bvadd (bvadd ?v_2 (_ bv4271429 32)) ?v_3) (_ bv7 32)) ?v_7))) (let ((?v_8 (bvadd (bvadd ?v_4 (_ bv3 32)) ?v_6))) (let ((?v_13 (bvadd (bvadd ?v_8 ?v_0) ?v_9))) (let ((?v_14 (bvsub ?v_13 (_ bv4271399 32)))) (let ((?v_16 (bvsub (bvadd (bvadd ?v_14 (_ bv4271400 32)) ?v_15) (_ bv1 32))) (?v_17 (bvadd ?v_14 (_ bv4271484 32))) (?v_10 (bvsub ?v_8 (_ bv4271399 32)))) (let ((?v_12 (bvsub (bvadd (bvadd ?v_10 (_ bv4271400 32)) ?v_9) (_ bv1 32))) (?v_11 (bvadd ?v_10 (_ bv4271478 32))) (?v_5 (bvsub ?v_4 (_ bv4271397 32))) (?v_1 (bvsub (bvadd (bvadd ?v_13 ?v_0) ?v_15) (_ bv4271399 32)))) (and true (= T2_20520 (bvor (bvshl ((_ zero_extend 8) T1_20521) (_ bv8 16)) ((_ zero_extend 8) T1_20520))) (= T2_20914 (bvor (bvshl ((_ zero_extend 8) T1_20915) (_ bv8 16)) ((_ zero_extend 8) T1_20914))) (bvule (bvsub (bvadd (bvadd ?v_1 (_ bv4271400 32)) ?v_19) (_ bv1 32)) (bvadd ?v_1 (_ bv4271420 32))) (not (= (bvadd (bvadd ?v_2 (_ bv29 32)) (_ bv4271400 32)) (_ bv0 32))) (not (= ?v_3 (_ bv0 32))) (bvult (bvadd ?v_5 (_ bv4271426 32)) (bvsub (bvadd (bvadd ?v_5 (_ bv4271400 32)) ?v_6) (_ bv1 32))) (= ?v_7 (_ bv1 32)) (not (= ?v_18 (_ bv0 32))) (bvule ?v_12 ?v_11) (not (= ?v_11 ?v_12)) (bvult (bvadd ?v_10 (_ bv4271462 32)) ?v_12) (not (= ?v_17 ?v_16)) (bvule ?v_16 ?v_17) (bvult (bvadd ?v_14 (_ bv4271460 32)) ?v_16) (bvult (bvadd ?v_14 (_ bv4271434 32)) ?v_16) (bvult (bvadd ?v_14 (_ bv4271418 32)) ?v_16) (= (bvadd (bvadd (bvadd (bvadd ?v_18 (bvsub (_ bv4294967295 32) ?v_6)) (bvsub (_ bv4294967295 32) ?v_9)) (bvsub (_ bv4294967295 32) ?v_15)) (bvsub (_ bv4294967295 32) ?v_19)) (_ bv0 32)))))))))))
(check-sat)

"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

To perform Semantic Fusion on the given formulas ω1 and ω2, we will follow the steps outlined in the task definition. Here are the detailed steps and the resulting fused formula:

### Step 1: Merge ω1 and ω2

First, we merge the two formulas into a single formula ω1 ∧ ω2, preserving original declarations and assertions.

### Step 2: Introduce a Fresh Variable z

We introduce a fresh variable `z` which is not present in either input formula. Since the formulas involve bit-vector operations, we will choose an appropriate bit-vector type for `z`. For simplicity, let's assume `z` is of type `(_ BitVec 32)`.

### Step 3: Choose the Fusion Function

Given that the formulas involve bit-vector operations, we need to choose an appropriate fusion function. For bit-vector operations, we can use a simple addition function:

\[ z = x + y \]

### Step 4: Define the Inversion Functions

Based on the fusion function \( z = x + y \), the inversion functions are:

\[ r_x(y, z) = z - y \]
\[ r_y(x, z) = 

In [ ]:
# Linear Integer Arithmetic (QF_LIA) dann das
formula1 = """
(set-logic QF_LIA)
(declare-fun x () Int)
(declare-fun y () Int)

(assert (> x 5))
(assert (< x 20))
(assert (> y (- x 3)))
(assert (< y (* 2 x)))
(check-sat)
"""

formula2 = """
(set-logic QF_LIA)
(declare-fun a () Int)
(declare-fun b () Int)

(assert (> a 0))
(assert (< a 50))
(assert (= (* a b) 100))
(assert (> b 1))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Bit-Vector Theory (QF_BV)
formula1 = """
(set-logic QF_BV)
(declare-fun x () (_ BitVec 8))
(declare-fun y () (_ BitVec 8))

(assert (= (bvadd x y) (_ bv25 8)))
(assert (bvult x (_ bv20 8)))
(assert (bvugt y (_ bv4 8)))
(check-sat)
"""

formula2 = """
(set-logic QF_BV)
(declare-fun p () (_ BitVec 8))
(declare-fun q () (_ BitVec 8))

(assert (= (bvmul p q) (_ bv40 8)))
(assert (bvult q (_ bv10 8)))
(assert (bvugt p (_ bv3 8)))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Nonlinear Integer Arithmetic (QF_NIA)
formula1 = """
(set-logic QF_NIA)
(declare-fun x () Int)
(declare-fun y () Int)

(assert (> x 2))
(assert (< x 10))
(assert (= (* x y) 20))
(assert (> y 1))
(check-sat)
"""

formula2 = """
(set-logic QF_NIA)
(declare-fun a () Int)
(declare-fun b () Int)

(assert (> a 3))
(assert (< a 15))
(assert (= (* a b) 30))
(assert (> b 2))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Arrays (QF_AUFBV)
formula1 = """
(set-logic QF_AUFBV)
(declare-const arr (Array Int Int))
(declare-const i Int)

(assert (> i 2))
(assert (= (select arr i) 42))
(assert (= (store arr 5 100) arr))
(check-sat)
"""

formula2 = """
(set-logic QF_AUFBV)
(declare-const arr2 (Array Int Int))
(declare-const j Int)

(assert (< j 10))
(assert (= (select arr2 j) 84))
(assert (= (store arr2 7 50) arr2))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Floating-Point Arithmetic (QF_FP)
formula1 = """
(set-logic QF_FP)
(declare-const x Float32)

(assert (fp.gt x (fp #b0 #b10000001 #b00000000000000000000000)))  ;; x > 2.0
(assert (fp.lt x (fp #b0 #b10000011 #b00000000000000000000000)))  ;; x < 8.0
(check-sat)
"""

formula2 = """
(set-logic QF_FP)
(declare-const y Float32)

(assert (fp.gt y (fp #b0 #b10000000 #b00000000000000000000000)))  ;; y > 1.0
(assert (fp.lt y (fp #b0 #b10000010 #b00000000000000000000000)))  ;; y < 4.0
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

In [ ]:
# Uninterpreted Functions (QF_UF)
formula1 = """
(set-logic QF_UF)
(declare-fun f (Int) Int)

(assert (= (f 1) 2))
(assert (= (f 2) 3))
(assert (= (f 3) 4))
(check-sat)
"""

formula2 = """
(set-logic QF_UF)
(declare-fun g (Int) Int)

(assert (= (g 4) 5))
(assert (= (g 5) 6))
(assert (= (g 6) 7))
(check-sat)
"""

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)